In [1]:
# not all imports are actually used

# built in python modules
import datetime
import os
import sys
import glob
import itertools
from collections import OrderedDict
from importlib import reload

# python add-ons
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib as mpl
import seaborn as sns
sns.set(style='whitegrid', rc={'figure.figsize':(12, 6)})

import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', 'raise')

import xarray as xr
import numexpr as ne

/home/travis/miniconda3/envs/py35Solar/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset = xr.open_dataset('/a2/uaren/2014/04/15/solar_3/wrfsolar_d02_hourly.nc')

In [3]:
def rh_calc(temps, pressure, qvapor):
    """
    Actually calculate rh from arrays
    """
    T1 = 273.16
    # Groff-Gratch equation
    ew = 10**(10.78574 * (1 - T1/temps)+
         - 5.028 * np.log10(temps/T1)+
         + 1.50475*10**-4 * (1 - 10**(-8.2969 * (temps/T1 - 1)))+
         + 0.42873*10**-3 * (10**(4.76955 * (1 - T1/temps)) - 1)+
         + 0.78614)
    # Bolton Equation
    #ew = ne.evaluate('6.112*exp(17.62*(temps-273.15)/(243.21 + temps - 273.15))')

    es = ew*(1.0016+3.15*10**-6*pressure-0.074/pressure)

    e = qvapor/(qvapor+0.62198)*pressure
    rh = e/es*100
    return rh

def dewpoint_calc(qvapor, pressure):
    """
    Actually calculate tdew from arrays
    """
    e = qvapor/(qvapor+0.62198)*pressure
    # Bolton's fit
    a = 6.112 #millibars
    b = 17.67
    c = 243.5 #deg C
    logterm = np.log(e/a)
    tdew = (c*logterm)/(b-logterm)
    return tdew

In [ ]:
dataset['pressure'] = dataset['P'] + dataset['PB']
dataset['temp'] = (dataset['T']+300)*(dataset['pressure']/100000)**0.2854
dataset['rh'] = rh_calc(dataset['temp'], dataset['pressure']/100, dataset['QVAPOR'])

In [12]:
dims = list(dataset['rh'].dims)
dims.remove('Time')
dims.remove('bottom_top')
dataset['average_rh'] = dataset['rh'].mean(dim=dims)

In [26]:
Times = dataset['Times'].to_pandas().apply(
    lambda x: pd.to_datetime(x.decode("utf-8").replace('_', ' ')))
Times = pd.Index(Times).tz_localize('UTC').tz_convert('MST')

In [70]:
dataset

<xarray.Dataset>
Dimensions:           (Time: 49, bottom_top: 37, south_north: 351, south_north_stag: 352, west_east: 429, west_east_stag: 430)
Coordinates:
    XLAT              (south_north, west_east) float32 30.9118 30.9114 ...
    XLONG             (south_north, west_east) float32 -115.175 -115.156 ...
  * Time              (Time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
  * bottom_top        (bottom_top) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...
  * south_north       (south_north) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...
  * west_east         (west_east) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...
  * west_east_stag    (west_east_stag) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ...
  * south_north_stag  (south_north_stag) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 ...
Data variables:
    P                 (Time, bottom_top, south_north, west_east) float32 1420.06 ...
    PB                (Time, bottom_top, south_north, west_east) float32 94855.1 ...
    QVAPOR            (T

In [73]:
wind_lats = dataset['XLAT'].values.ravel()
wind_lons = dataset['XLONG'].values.ravel()
U_shape = dataset['U'].isel(Time=0, bottom_top=0).shape
V_shape = dataset['V'].isel(Time=0, bottom_top=0).shape

In [64]:
num_of_times = (Times <= 
                pd.to_datetime('2014-04-15 20:00:00').tz_localize('MST')).sum()
Times = Times[slice(0, num_of_times)]

temp = dataset.isel(Time=0)
bottom_top[Times[0]] = temp['bottom_top'].where(
    temp['average_rh'] == temp['average_rh'].max(), drop=True).item()
U = pd.DataFrame(
    data=temp['U'].sel(bottom_top = bottom_top[Times[0]]).values.ravel()[None, :],
    index=[Times[0]])
V = pd.DataFrame(
    data=temp['V'].sel(bottom_top = bottom_top[Times[0]]).values.ravel()[None, :],
    index=[Times[0]])

In [65]:
for t in np.arange(num_of_times - 1) + 1:
    temp = dataset.isel(Time=t)
    bottom_top[Times[t]] = temp['bottom_top'].where(
        temp['average_rh'] == temp['average_rh'].max(), drop=True).item()
    U_temp = pd.DataFrame(
        data=temp['U'].sel(bottom_top = bottom_top[Times[t]]).values.ravel()[None, :],
        index=[Times[t]])
    V_temp = pd.DataFrame(
        data=temp['V'].sel(bottom_top = bottom_top[Times[t]]).values.ravel()[None, :],
        index=[Times[t]])
    U = U.append(U_temp)
    V = V.append(V_temp)

In [80]:
U.to_hdf('./data/winds/U_15.h5', 'U_15')
V.to_hdf('./data/winds/V_15.h5', 'V_15')
bottom_top.to_hdf('./data/winds/bottom_top_15.h5', 'bottom_top_15')
np.save('./data/winds/wind_lats', wind_lats)
np.save('./data/winds/wind_lons', wind_lons)

In [86]:
np.save('./data/winds/U_shape', U_shape)
np.save('./data/winds/V_shape', V_shape)
Times.to_series().to_hdf('./data/winds/times.h5', 'times')